#Set up

In [1]:
!pip install evaluate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, DataCollatorWithPadding, Trainer, pipeline, BertTokenizer, BertForSequenceClassification
from datasets import load_dataset, Dataset, ClassLabel
from google.colab import files, drive
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve
import matplotlib.pyplot as plt
import torch
import evaluate
import numpy as np
import pandas as pd
import os

# Read preprocessing email data

In [3]:
uploaded = files.upload()

Saving preprocessed_test_emails.csv to preprocessed_test_emails.csv


In [ ]:
# import prepeocessed emails data
dataset = pd.read_csv("preprocessed_emails.csv")
dataset = dataset[dataset['text'].notna() & (dataset['text'].str.strip() != '')]
dataset = Dataset.from_pandas(dataset)

test_set = pd.read_csv("preprocessed_test_emails.csv")
test_set = test_set[test_set['text'].notna() & (test_set['text'].str.strip() != '')]
test_set = Dataset.from_pandas(test_set)

# Main body

In [ ]:
# disable tokenizers parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# disable wandb
os.environ["WANDB_MODE"] = "disabled"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# Training the model
id2label = {0: "ham", 1: "phishing"}
label2id = {"ham": 0, "phishing": 1}

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
# Bring BERT base model from HuggingFace
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id)

# tokenize text and truncate sequences to be no longer than BERT's maximum input length
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# define ClassLabel features
labels = ClassLabel(num_classes=2, names=["ham", "phishing"])

# update tokenized dataset
tokenized_dataset = tokenized_dataset.cast_column("label", labels)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluation metrics
metrics = evaluate.combine(["accuracy", "precision", "recall", "ealvaradob/false_positive_rate"])

# Calculate the accuracy, recall, and FPR
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metrics.compute(predictions=predictions, references=labels)

# Split the data
train_test = tokenized_dataset.train_test_split(test_size=0.2, seed=42, stratify_by_column='label')
train_dataset = train_test['train']
test_dataset = train_test['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir="bert-large-finetuned-phishing",
    run_name="my-phishing-detection-run",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    torch_compile=False,
    fp16=True,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Store fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('bert-large-finetuned-phishing')
tokenizer = AutoTokenizer.from_pretrained('bert-large-finetuned-phishing')

# Save model to Google Drive
drive.mount('/content/drive')
save_path = "/content/drive/MyDrive/CIS_6010/bert-large-finetuned-phishing"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model and tokenizer saved successfully!")

# Test

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_test_set = test_set.map(preprocess_function, batched=True)

# define ClassLabel features if needed
labels = ClassLabel(num_classes=2, names=["ham", "phishing"])
tokenized_test_set = tokenized_test_set.cast_column("label", labels)

predictions = trainer.predict(tokenized_test_set)

predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids
# Calculate the accuracy, recall, and FPR
test_metrics = metrics.compute(predictions=predicted_labels, references=true_labels)
print(test_metrics)

Map:   0%|          | 0/4145 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4145 [00:00<?, ? examples/s]

{'accuracy': 0.9983112183353438, 'precision': 0.9978479196556671, 'recall': 0.9971326164874552, 'false_positive_rate': 0.001090909090909091}


In [ ]:
text = (
    "Text: Hi, "
          "Thanks for working with us. 28 Aug 2016. "
          "If you’ve already, please ignore this email and sorry for bothering you. If you’ve not, please do so as soon as possible."
          "To view your visit "
          "If you’ve got any questions, or want to arrange alternative don’t hesitate to get in touch."
          "Thanks, NJW Limited"
)

In [ ]:
classifier = pipeline("text-classification", model = model, tokenizer = tokenizer)
classifier(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'phishing', 'score': 0.9996509552001953}]

# Model Evaluation (Not included)

This part doesn't count into the model training and model evaluation parts. Please read final report for explantions.

In [4]:
drive.mount('/content/drive')
model_path = '/content/drive/My Drive/CIS_6010/bert-large-finetuned-phishing'

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# change to evaluation mode
model.eval()

test_set = pd.read_csv("preprocessed_test_emails.csv")
test_set = test_set[test_set['text'].notna() & (test_set['text'].str.strip() != '')]
test_set = Dataset.from_pandas(test_set)

X_test = test_set['text']
y_true = test_set['label']

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("/content/drive/My Drive/CIS_6010/bert-large-finetuned-phishing")
inputs = tokenizer(list(X_test), return_tensors="pt", padding=True, truncation=True, max_length=512)

Mounted at /content/drive


In [ ]:
# use model to predict
with torch.no_grad():
    outputs = model(**inputs)

# get predictions
logits = outputs.logits
y_pred = logits.argmax(dim=1).tolist()  # 转为 Python 列表
print("Predicted labels:", y_pred)

# calculate model performence
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, logits.softmax(dim=1)[:, 1])  # 使用概率计算 AUC

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)

# classification report
report = classification_report(y_true, y_pred, target_names=["Ham", "Spam"])
print("Classification Report:\n", report)